In [1]:
import openai
import pandas as pd
import numpy as np
import re
from datetime import datetime
import pathlib
from pathlib import Path
import json
import os
import tiktoken
import summary_scripts as ss

In [2]:
import openai
from openai import OpenAI

In [ ]:
# client = OpenAI(api_key='<your api_key>')

In [3]:
COMPLETIONS_MODEL = "gpt-3.5-turbo-1106"

In [4]:
SEPARATOR = "\n* "
ENCODING = "cl100k_base"  # encoding scheme for text-davinci-003, the embedding model we are going to use

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

def add_token_counts(df,src:str,targ:str):
    df[targ] = df[src].apply(lambda x:len(encoding.encode(str(x))))
    return df
MAX_CONTEXT = 500

In [5]:
### read in es.txt file
def read_lines(file):
    with open(file) as f:
        lines = [l.strip() for l in f.readlines() if '---  page ' not in l and not l.strip() == '']
    return lines

def read_txt(file):
    with open(file) as f:
        lines = [l.strip() for l in f.readlines() if '---  page ' not in l and not l.strip() == '']
        txt = ''.join(lines)
    return txt

In [12]:
role_system_bullets = """
You are a multi-lingual international lawyer whos legal specialty is data protection and processing.
Additionally, you know and work with the General Data Protection Regulations (GDPR) of European Union.
You will produce a summary of the document from the data protection authority by answering questions. 
The summary will be in English only.
As a general rule, be clear and precise about your writing. If the answer is unclear, explain how so.  
"""

In [13]:
prpt_bullets = """"
Please finish the task one by one.
1. In one or two sentences, identify the type of the document.
2. Identify the complainant and defendant mentioned in the document, if any. Determine the type of complainant and the type of defendant as individual, or company, or public institution, or public authority. Describe the relationship between these two parties if possible. 
3. Please elaborate the subject matter of the dispute between these two parties, and then explain the data involved in this dispute.
4. What articles of GDPR, if any, does the complainant claim that the defendant potentially violates?
5. Does the legal authority dismiss the submitted complaint or case? 
6. What is the outcome of the dispute? From the final resolution in the document, describe all the enforcements by the authority in detail (including but not limited to: fine, warning, investgation and other penalties or obligations)
7. What facts and evidence are verified or proved by the authority?
8. In case the complaint is approved or the authority releases the enforcement, identify the GDPR articles violated by the defendant. Your answer should only consider the decision made by the data protection authority, together with the verified evidence or facts in the previous steps.
9. Is the document a final decision, or a preliminary decision (e.g. an order to stop data processing until a final decision is made), or an intermediate procedural step (e.g. a request for additional information), or related to a previous procedure, or something else?

Here is the document:

"""

In [14]:
prpt_bullets_ada = """"
Please finish the tasks one by one.
1. In one or two sentences, identify the type of the document.
2. Identify the complainant and defendant mentioned in the document, if any. Determine the type of complainant and the type of defendant as individual, or company, or public institution, or public authority. Describe the relationship between these two parties if possible. 
3. Please elaborate the subject matter of the dispute between these two parties, and then explain the data involved in this dispute.
4. What kind of data processing was involved (including but not limited to collection, storage, transfer, publication, dissimation, etc.) and for what purpose (including but not limited to marketing, journalistic purposes, market research, etc.)?
5. What articles of GDPR, if any, does the complainant claim that the defendant potentially violates?
6. Does the legal authority dismiss the submitted complaint or case? 
7. What is the outcome of the dispute? From the final resolution in the document, describe all the enforcements by the authority in detail (including but not limited to: fine, warning, investgation and other penalties or obligations)
8. What facts and evidence are verified or proved by the authority?
9. In case the complaint is approved or the authority releases the enforcement, identify the GDPR articles violated by the defendant, if any. Your answer should only consider the decision made by the data protection authority, together with the verified evidence or facts in the previous steps.
10. Is the document a final decision, or a preliminary decision (e.g. an order to stop data processing until a final decision is made), or an intermediate procedural step (e.g. a request for additional information), or related to a previous procedure, or something else?

Here is the document:

"""

In [15]:
prpt_long_summ_ada = """ 
Follow the below instructions to produce a concise and informative summary (between 5 and 10 paragraphs) in English of the attached document.

Return 1 paragraph or more about : What type of legal document is this (be precise)?

Return 1 paragraph or more about : Who is the complainant? If no complainant is found, is the DPA (data protection authority) the prosecutor itself? 
Is there additional information about the complainant (e.g. is it an individual, a company, a public institution, a public authority, etc.)?
Who is the defendant (i.e. the accused party)? Is there additional information about the defendant (e.g. is it an individual, a company, a public institution, a public authority, etc.)? 
If the defendant or complainant is a company, what is its business sector? If the defendant or complainant is a public institution or public authority, what is its role (e.g. hospital, school, university, police, municipality, etc.)?

Return 1 or 2 paragraphs about : explain what the complaint is about, and then elaborate the subject matter of the dispute between these two parties, and then explain the data involved in this dispute.
From that, explain what kind of behavior gave rise to the dispute (e.g. data breach, illegal data collection, missing consent to data retention/transfer/publication, etc.).

Return 1 paragraph or more about : What type of data was involved (e.g. medical data, dashboard footage, video surveillance)? 
What kind of data processing was involved (e.g. collection, storage, transfer, publication, dissimation, etc.) and for what purpose (e.g. marketing, journalistic purposes, market research, etc.)?

Return 1 paragraph or more about : What was the outcome of the dispute, i.e. what is the decision of the data protection authority (e.g. fine, warning, order, investigation, dismissal, termination of procedure, etc.)?
Was a fine issued, and if so, what was the amount of the fine? Was an order or warning issued, and if so, what was the exact order or warning?

Return 1 paragraph or more about : Is the document a final decision, or a preliminary decision (e.g. an order to stop data processing until a final decision is made), or an intermediate procedural step (e.g. a request for additional information), 
or related to a previous procedure, or something else?

The document is here:

"""

In [16]:
prpt_long_summ = """ 
Follow the below instructions to produce a concise and informative summary (between 5 and 10 paragraphs) in English of the attached document.

Return 1 paragraph or more about : What type of legal document is this (be precise)?

Return 1 paragraph or more about : Who are the parties to the dispute? Is there a complainant (if so, what is its name) or did the DPA (data protection authority) initiate the procedure? 
Is there additional information about the complainant (e.g. is it an individual, a company, a public institution, a public authority, etc.)?
What is the name of the defendant (i.e. the accused party)? Is there additional information about the defendant (e.g. is it an individual, a company, a public institution, a public authority, etc.)? 
If the defendant or complainant is a company, what is its business sector? If the defendant or complainant is a public institution or public authority, what is its role (e.g. hospital, school, university, police, municipality, etc.)?

Return 1 paragraph or more about :  What is the subject matter of the dispute (e.g. what did the defendant do that gave rise to this legal procedure? etc.)?
What kind of incident gave rise to the dispute (e.g. data breach, illegal data collection, missing consent to data retention/transfer/publication, etc.)?
What are the alleged violations of the GDPR or national data protection laws (e.g. violation of the right to confidentiality, violation of the right to information, violation of the right to deletion, etc.)?

Return 1 paragraph or more about : What type of data was involved (e.g. medical data, dashboard footage, video surveillance)? 
What kind of data processing was involved (e.g. collection, storage, transfer, publication, dissimation, etc.) and for what purpose (e.g. marketing, journalistic purposes, market research, etc.)?

Return 1 paragraph or more about : What was the outcome of the dispute, i.e. what is the decision of the data protection authority (e.g. fine, warning, order, investigation, dismissal, termination of procedure, etc.)?
Was a fine issued, and if so, what was the amount of the fine? Was an order or warning issued, and if so, what was the exact order or warning?

Return 1 paragraph or more about : Is the document a final decision, or a preliminary decision (e.g. an order to stop data processing until a final decision is made), or an intermediate procedural step (e.g. a request for additional information), 
or related to a previous procedure, or something else?

The document is here:

"""

In [51]:
summary_ex1_spain = """
Summary:
1. The document is a "Resolution of Termination of Procedure by Voluntary Payment" issued by the Spanish Data Protection Agency regarding a complaint against a specific foundation for alleged violations of data protection regulations.
2. The complainant is an individual named A.A.A., while the defendant is the FUNDACIÓN ESPAÑOLA DE MEDICINA ESTÉTICA Y LONGEVIDAD, a company involved in medical and longevity training (educational institution). The relationship exists as the complainant has raised concerns about the defendant's data processing practices.
3. The dispute involves the defendant allegedly creating WhatsApp and Instagram groups related to a Master's program, exposing participants' personal data. The data involved include participants' personal information, correspondences, and screenshots from social media.
4. The data processing involved data collection through the registration form, storage on the defendant's website, and potential dissemination on social media for educational purposes.
5. Articles 13 and 7 of the General Data Protection Regulation (GDPR) are potentially claimed to be violated by the defendant.
6. The legal authority has initiated a sanctioning procedure against the defendant.
7. The outcome includes the defendant voluntarily paying a fine of 2400 euros for the violations. The enforceable measures include the imposition of a financial penalty and the recognition of responsibility by the defendant.
8. The facts verified by the authority include the lack of an updated privacy policy and the absence of explicit consent for data processing.
9. The GDPR articles violated by the defendant are 13 and 7.
10. The document represents a final resolution, as the defendant has recognized its responsibility and voluntarily paid the imposed fine.
"""
target_out_1 = """
{
  "complainant": {
    "complainant_yn": "yes",
    "complainant_name": "A.A.A.",
    "complainant_category": "individual"
  },
  "defendant": {
    "defendant_yn": "yes",
    "defendant_name": "FUNDACIÓN ESPAÑOLA DE MEDICINA ESTÉTICA Y LONGEVIDAD",
    "defendant_category": "company",
    "defendant_sector": "medicine and health"
  },
  "dispute": {
    "dispute_one_liner": "The dispute involves the defendant allegedly creating WhatsApp and Instagram groups related to a Master's program, exposing participants' personal data.",
    "violation_claimed_by_complainant": ["Articles 13 and 7 of the GDPR"]
  },
  "data": {
    "data_one_liner": "Data involved include participants' personal information, correspondences, and screenshots from social media.",
    "data_type_list": ["personal information"],
    "data_handling_list": ["data collection", "storage", "dissemination"],
    "purpose_of_data_handling_list": ["educational purposes"]
  },
  "outcome": {
    "outcome_one_liner": "The defendant voluntarily paid a fine of 2400 euros for the violations. The enforceable measures include the imposition of a financial penalty and the recognition of responsibility by the defendant.",
    "outcome_type_list": ["fine"],
    "fine_amount": 2400,
    "currency": "euros"
  },
  "violation_of_GDPR_based_on_authority": ["Articles 13 and 7"],
  "document_type": "resolution of termination of procedure by voluntary payment"
"""
summary_ex2_spain = """
1. The document is a "RESOLUCIÓN DE PROCEDIMIENTO SANCIONADOR" - a resolution of a sanctioning procedure by the Spanish Data Protection Agency.

2. The complainants are D. A.A.A. and D. B.B.B. The defendant mentioned in the document is D. C.C.C. The complainants are individuals, and the defendant is another individual.

3. The dispute revolves around the publication of a news article on the website eltaquigrafo.com by the defendant, which mentioned and made public personal data of the complainants without their consent or authorization. The personal data involved in this dispute are the complete names and D.N.I. (National Identity Document) of the complainants.

4. The data processing involved was the publication of personal data on a website without consent or authorization, for the purpose of making it public to all internet users without any restriction.

5. The complainant claims that the defendant potentially violates Article 5.1.c) of the General Data Protection Regulation (GDPR).

6. The legal authority did not dismiss the submitted complaint and proceeded with the sanctioning procedure.

7. The outcome of the dispute is that the defendant is imposed with a warning as well as a requirement to remove the complainants' DNI from the publication and provide evidence of compliance.

8. The authority verified and proved that the defendant published the complainants' personal data without their consent or authorization.

9. The GDPR Article violated by the defendant is Article 5.1.c), as determined by the data protection authority, based on the verified evidence and facts.

10. The document is a final decision in the sanctioning procedure.
"""
target_out_2 = """
{
  "complainant": {
    "complainant_yn": "yes",
    "complainant_name": ["D. A.A.A.", "D. B.B.B."],
    "complainant_category": "individuals"
  },
  "defendant": {
    "defendant_yn": "yes",
    "defendant_name": "D. C.C.C.",
    "defendant_category": "individual"
  },
  "dispute": {
    "dispute_one_liner": "The dispute revolves around the publication of a news article on the website eltaquigrafo.com by the defendant, which mentioned and made public personal data of the complainants without their consent or authorization.",
    "violation_claimed_by_complainant": ["Article 5.1.c) of the GDPR"]
  },
  "data": {
    "data_one_liner": "The personal data involved in this dispute are the complete names and D.N.I. (National Identity Document) of the complainants.",
    "data_type_list": ["complete names", "D.N.I."],
    "data_handling_list": ["publication without consent or authorization"],
    "purpose_of_data_handling_list": ["making it public to all internet users without any restriction"]
  },
  "outcome": {
    "outcome_one_liner": "The defendant is imposed with a warning as well as a requirement to remove the complainants' DNI from the publication and provide evidence of compliance.",
    "outcome_type_list": ["warning", "order"],
    "warning": "The defendant is warned for publishing personal data without consent.",
    "order": "The defendant is required to remove complainants' DNI from the publication and provide evidence of compliance."
  },
  "violation_of_GDPR_based_on_authority": ["Article 5.1.c)"],
  "document_type": "final decision in the sanctioning procedure"
}
"""
summary_ex3_spain = """
1. The document is a "RESOLUCIÓN DE PROCEDIMIENTO SANCIONADOR" (Sanctioning Procedure Resolution) issued by the Agencia Española de Protección de Datos (Spanish Data Protection Agency) against the entity SOTA S.C.P. regarding a potential breach of the General Data Protection Regulation (GDPR) of the European Union.

2. The complainant is D. A.A.A., an individual, and the defendant is the entity SOTA S.C.P., a company. The relationship between these parties is that the complainant has filed a complaint against the defendant for non-compliance with data protection regulations.

3. The dispute concerns the defendant's website www.joiestelada.cat, dedicated to online product sales, allegedly violating data protection regulations by lacking a proper Privacy Policy, including a contact form that collects personal data without providing necessary information, and not having a link to the Privacy Policy or a checkbox for users to explicitly accept it. The data involved includes personal data collected through the website's contact form.

4. The data processing involved includes the collection of personal data through the website's contact form without providing necessary information or obtaining explicit consent from the users. The purpose of this processing is for online product sales. 

5. The complainant claims that the defendant potentially violates Article 13 of the GDPR.

6. The legal authority initiates a sanctioning procedure against the defendant.

7. The outcome of the dispute is that the defendant is "APERCIBIDO" (warned) and required to adjust its Privacy Policy within one month, failing which it may face a fine for infringing Article 13 of the GDPR. This enforcement includes a warning and an obligation to comply with data protection regulations.

8. The verified fact is that the defendant's website lacks a Privacy Policy and fails to provide necessary information and consent options when collecting users' personal data.

9. The GDPR article violated by the defendant, as identified by the Data Protection Authority, is Article 13.

10. The document is a final decision that puts an obligation on the defendant to comply with the GDPR within a specified period, failing which it may face further enforcement measures.
"""
target_out_3 = """
{
  "complainant": {
    "complainant_yn": "yes",
    "complainant_name": "D. A.A.A.",
    "complainant_category": "individual"
  },
  "defendant": {
    "defendant_yn": "yes",
    "defendant_name": "SOTA S.C.P.",
    "defendant_category": "company"
  },
  "dispute": {
    "dispute_one_liner": "The dispute concerns the defendant's website www.joiestelada.cat, dedicated to online product sales, allegedly violating data protection regulations by lacking a proper Privacy Policy, including a contact form that collects personal data without providing necessary information, and not having a link to the Privacy Policy or a checkbox for users to explicitly accept it.",
    "violation_claimed_by_complainant": ["Article 13 of the GDPR"]
  },
  "data": {
    "data_one_liner": "The data involved includes personal data collected through the website's contact form.",
    "data_type_list": ["personal data"],
    "data_handling_list": ["collection"],
    "purpose_of_data_handling_list": ["online product sales"]
  },
  "outcome": {
    "outcome_one_liner": "The defendant is 'APERCIBIDO' (warned) and required to adjust its Privacy Policy within one month, failing which it may face a fine for infringing Article 13 of the GDPR. This enforcement includes a warning and an obligation to comply with data protection regulations.",
    "outcome_type_list": ["warning", "order or obligations"],
    "warning": "The defendant is warned for violating data protection regulations.",
    "order": "The defendant is required to adjust its Privacy Policy within one month."
  },
  "violation_of_GDPR_based_on_authority": ["Article 13"],
  "document_type": "final decision in the sanctioning procedure"
}
"""
summary_ex4_spain = """
This document is a final decision by the Spanish Data Protection Authority resolving a complaint and imposing a penalty on an individual for a data protection violation.

The complainant in this case is the Spanish Data Protection Authority, and the defendant is an individual, A.A.A. The dispute is related to the dissemination of a video depicting a violent incident involving a woman and a child through social media platforms.

The dispute involves the dissemination of a video depicting a violent incident involving a woman and her child, the publication of which raised concerns about potential violations of data protection laws.

The data processing involved the collection, dissemination, and publication of personal images and information. The purpose was to create awareness about violence against women. Articles 6 and 83.5 of the GDPR are relevant to the complaint.

The legal authority did not dismiss the submitted complaint or case.

The outcome of the dispute is the imposition of a 10,000 euro fine on the defendant for violating Article 6.1 of the GDPR. The authority verified that the processing of personal data without a lawful basis occurred, leading to a violation of the GDPR.

The GDPR articles violated by the defendant are Article 6.1, leading to the imposition of a 10,000 euro fine. The evidence presented by the authority supported the violation of this article.

The document is a final decision by the Spanish Data Protection Authority, imposing a penalty on the defendant for a data protection violation.
"""
target_out_4 = """
{
  "complainant": {
    "complainant_yn": "yes",
    "complainant_name": "Spanish Data Protection Authority",
    "complainant_category": "public authority",
    "complainant_role": null
  },
  "defendant": {
    "defendant_yn": "yes",
    "defendant_name": "A.A.A.",
    "defendant_category": "individual"
  },
  "dispute": {
    "dispute_one_liner": "The dispute involves the dissemination of a video depicting a violent incident involving a woman and her child, the publication of which raised concerns about potential violations of data protection laws.",
    "violation_claimed_by_complainant": ["Articles 6 and 83.5 of the GDPR"]
  },
  "data": {
    "data_one_liner": "The data processing involved the collection, dissemination, and publication of personal images and information.",
    "data_type_list": ["personal images", "personal information"],
    "data_handling_list": ["collection", "dissemination", "publication"],
    "purpose_of_data_handling_list": ["create awareness about violence against women"]
  },
  "outcome": {
    "outcome_one_liner": "The outcome of the dispute is the imposition of a 10,000 euro fine on the defendant for violating Article 6.1 of the GDPR. The authority verified that the processing of personal data without a lawful basis occurred, leading to a violation of the GDPR.",
    "outcome_type_list": ["fine"],
    "fine_amount": 10000,
    "currency": "euro"
  },
  "violation_of_GDPR_based_on_authority": ["Article 6.1"],
  "document_type": "final decision with penalty imposition"
}
"""

In [52]:
msg_sys_json = """ You are a JSON representation expert with legal expertise. You will be given a JSON schema (with examples and pseudo code) and later a summary of a legal decision document. Your job is to follow this JSON schema and generate a JSON file from the summary. Be careful not to be limited your writing to the given examples(e.g.) in the schema. Only use them if they fit the summary. However, you have to follow the schema rigorously.
The JSON schema is here:
{ "complainant" : {
        complainant_yn (string) : "yes or no", depending on whether a complainant other than the data protection authority is mentioned in the text
        if complainant_yn == "yes":
            complainant_name (string) : "name of the complainant",
            complainant_category (string): "individual, company, public authority, or public institution",
            if complainant_category == "company":
                complainant_sector (string) : "business sector of the complainant"
            if complainant_category == "public authority" or "public institution":
                complainant_role (string): "role of the complainant" (e.g. "hospital", "school", "police", "municipality", etc.)
        if complainant_yn == "no":
            initiator_is_DPA_yn (string): "yes or no", depending on whether or not the data protection authority initiated the procedure
            if initiator_is_DPA_yn == "yes":
                initiating_reason (string): "reason for initiating the procedure" (e.g. report/complaint by a unnamed third party, report by defendant, etc.)
    },
    defendant : {
        defenant_yn (string): "yes or no", depending on whether a defendant is mentioned in the text
        if defendant_yn == "yes":
            defendant_name (string): "name of the defendant",
            defendant_category (string): "individual, company, public authority, or public institution",
            if defendant_category == "company":
                defendant_sector (string) : "business sector of the defendant"
            if defendant_category == "public authority" or "public institution":
                defendant_role (string): "role of the defendant" (e.g. "hospital", "school", "police", "municipality", etc.)
    }, 
    dispute : {
        dispute_one_liner (string) : "Answer in one sentence: What the dispute/procedure is about, i.e. what did the defendant do that caused the dispute/procudure?"
        violation_claimed_by_complainant (list of strings): list of alleged violations of provisions of the GDPR or national data protection laws by the complainant
    }, 
    data : {
        data_one_liner (string) : "Answer in one sentence: What type of data was involved?"
        data_type_list (list of strings): list of data types involved
        data_handling_list (list of strings): list of data handling categories involved (e.g. ["data breach", "collection", "retention", "dissemination", “corruption”, ”transfer", etc.])
        purpose_of_data_handling_list (list of strings): list of purposes of data handling (e.g. ["marketing", "journalistic purposes", "market research", etc.])
    }, 
    outcome : {
        outcome_one_liner (string) : "Answer in one sentence: What was the outcome of the dispute/procedure, i.e. what is the decision of the data protection authority?"
        outcome_type_list (list of strings): list of outcome types (e.g. ["fine", "warning", "order", "investigation", "dismissal", etc.])
        if outcome_type_list contains "fine":
            fine_amount (int): "amount of the fine"
            currency (string): "currency of the fine"
        if outcome_type_list contains "order":
            order (string): "What was the exact order?"
        if outcome_type_list contains "warning":
            warning (string): "What was the exact warning?"
    },
"violation_of_GDPR_based_on_authority": ["list of violated GDPR articles by the defendant, as identified by the Data Protection Authority, if applicable”]
,
    document_type (string): "What type of legal document is this (be precise, e.g. injunction order, administrative sanctioning, temporary order to halt data processing, warning, request for information)?"
}
"""

In [48]:
examples_summary = [summary_ex1_spain.replace('\n', ' '),summary_ex2_spain.replace('\n', ' '),summary_ex3_spain.replace('\n', ' '),summary_ex4_spain.replace('\n', ' ')]
examples_json = [target_out_1.replace('\n', ' '),target_out_2.replace('\n', ' '),target_out_3.replace('\n', ' '),target_out_4.replace('\n', ' ')]

In [19]:
COMPLETIONS_MODEL = "gpt-3.5-turbo-1106"
MAX_TOKEN_SUMMARY = 1000
MAX_TOKEN_DOC = 14000
TEMPERATURE = 1.0

In [20]:
def summary_bullets_gpt(prpt,txt,msg,
                    show_prompt: bool = False):
    
    messages = [{"role" : "system", "content": msg}]
    
    if show_prompt:
        print(prpt)


    messages.append({"role" : "user", "content":prpt + txt})
    response = client.chat.completions.create(
        model=COMPLETIONS_MODEL,
        messages=messages,
        max_tokens=MAX_TOKEN_SUMMARY,
        temperature = TEMPERATURE
        )

    return '\n' + response.choices[0].message.content

In [24]:
def get_completion(doc,prompt, msg_sys, country = "spain", modelname = "gpt-3.5-turbo-1106", temp = 1.0,max_tokens = 1000):
    mssges = [{"role" : "system", "content": msg_sys}]
    
    # if show_prompt:
    #     print(prompt)
    mssges.append({"role" : "user", "content":prompt + doc})
    completion = client.chat.completions.create(
        model=modelname,
        messages= mssges,
        max_tokens=max_tokens,
        temperature = temp)

    #print tokens used in prompt and completion
    print("Tokens in prompt: ", completion.usage.prompt_tokens)
    print("Tokens in completion: ", completion.usage.completion_tokens)

    return completion

In [25]:
def write_summary(doc,savedir,country,save_result = True,token_limit = 14000, model_name = "gpt-3.5-turbo-1106", t = 1.0):
    '''
    Summarizes the text in the file at filepath and saves the summary and completion object to savedir
    filepath: path to text file
    savedir: directory where to save the summary and completion object
    country: country for which to summarize
    token_limit: maximum number of tokens for the text
    modelname: name of the model to use

    Returns summary and completion object
    '''

    # if filepath.endswith(".html"):
    #     text = html2str(filepath)
    #     savepath_original = os.path.join(savedir, "original.html")

    #     #save the original file in savedir if it does not exist yet
    #     if not os.path.exists(savepath_original):
    #         with open(filepath, 'r', encoding="utf-8") as f:
    #             original_text = f.read()

    #         with open(savepath_original, "w") as f:
    #             f.write(original_text)

    # else: raise NotImplementedError(f"Filetype {filepath} not implemented")
    
    #count text tokens
    tokens_text = ss.count_tokens(doc, modelname=model_name)

    if tokens_text >= token_limit:
        print("Tokens in text: ", tokens_text)
        print(f"Text too long: Longer than {token_limit} tokens. Returning empty string and empty dictionary.")
        return '' , {}
    
    completion = get_completion(doc,prpt_bullets_ada, role_system_bullets, country,modelname = model_name, temp = t) ### to-do: write get completion
    summary = completion.choices[0].message.content

    completion_dict = ss.convert_chat_completion_to_dict(completion)

    if save_result:

        savepath_summary = os.path.join(savedir, "summary_t" + str(t) + ".txt")
        savepath_completion = os.path.join(savedir, "completion.json")

        completion_json = json.dumps(completion_dict)

        #save the summary and the completion object in the savedir

        with open(savepath_completion, "w") as f:  
            f.write(completion_json)

        with open(savepath_summary, "w") as f:
            f.write(summary)
    
    return summary, completion_dict

In [35]:
def construct_message_summary_to_json(summary,msg_sys,example_inputs = [], example_outputs = []):
    message = []
    message.append({"role": "system", "content": msg_sys})

    for i in range(len(example_inputs)):
        message.append({"role": "user", "content": example_inputs[i]})
        message.append({"role": "assistant", "content": example_outputs[i]})

    message.append({"role": "user", "content": summary})

    return message

In [63]:
def summary_to_json(mssges, savedir, modelname = "gpt-3.5-turbo-1106", save_json = True, temp = 1.0, max_tokens = 500):
    
    completion = client.chat.completions.create(
            model=modelname,
            response_format={ "type": "json_object" },
            messages= mssges,
            max_tokens= max_tokens,
            temperature=temp)
    
            #print tokens used in prompt and completion
    print("Tokens in prompt: ", completion.usage.prompt_tokens)
    print("Tokens in completion: ", completion.usage.completion_tokens)

    try:
        response_dict = json.loads(completion.choices[0].message.content.replace('\n',''))
    except Exception as e:
        print(e)
        print("API response could not be converted to dictionary. Returning the content of the completion message and the completion object as dict")
        return completion.choices[0].message.content, ss.convert_chat_completion_to_dict(completion)
    
    if save_json:
        savepath_json = savedir + '/tj' + str(temp) +".json"
        with open(savepath_json, "w") as f:
            json.dump(response_dict, f)

    return response_dict, ss.convert_chat_completion_to_dict(completion)

In [26]:
SRC_DIR = './DPA-decisions/DPA-decisions/SPAIN/' #full path to source directory
TRG_DIR = './spain_qr_results/'#full path to target directory

In [27]:
TOTAL_NUM_SPAIN = 1238

In [65]:
# c_not_summarised = 0
# for root, dirs, files in os.walk(SRC_DIR):
#     for f in files:
#         if f.endswith(".pdf.es.txt"):
#             filepath = os.path.join(root, f)
#             txt = read_txt(filepath)
#             path = pathlib.PurePath(filepath)
#             parent = path.parent.name
#             grandpa = path.parent.parent.name
#             dir_grand  = TRG_DIR+grandpa
#             dir_parent = TRG_DIR+grandpa+"/"+parent
#             if grandpa == "RESOLUCION_DE_PROCEDIMIENTO_SANCIONADOR":
#                 tokens_text = ss.count_tokens(txt, modelname=COMPLETIONS_MODEL)
#                 if tokens_text >= 14000:
#                     c_not_summarised += 1

# print(c_not_summarised)

78


In [28]:
path_src_list  = [] # collection of paths to all case input folders
dir_dst_list = [] #  collection of paths to all case output folders 
c = 0
for root, dirs, files in os.walk(SRC_DIR):
    for f in files:
        if f.endswith(".pdf.es.txt"):
            filepath = os.path.join(root, f)
            txt = read_txt(filepath)
            path = pathlib.PurePath(filepath)
            parent = path.parent.name
            grandpa = path.parent.parent.name
            dir_grand  = TRG_DIR+grandpa
            dir_parent = TRG_DIR+grandpa+"/"+parent
            if not os.path.exists(dir_grand):
                os.makedirs(dir_grand)
            if not os.path.exists(dir_parent):
                os.makedirs(dir_parent)
                ### produced summary (completion dict + summary.txt)
                ### 
                ### change summary to json (need to prepare 3-4 examples.)
            if grandpa == "RESOLUCION_DE_PROCEDIMIENTO_SANCIONADOR":
                c +=1
                # print(filepath)
                # print(dir_parent)
                # print(txt)
                # sum_path = dir_parent + '/summary.txt'
                doc_path = dir_parent + '/document.txt'
                # if not os.path.exists(doc_path):
                with open(filepath, 'r', encoding="utf-8") as f:
                    original_text = f.read()

                with open(doc_path, "w") as f:
                    f.write(original_text)
                
                summary, completion_dict = write_summary(txt,dir_parent,country="spain", save_result = True,token_limit = 14000, model_name = "gpt-3.5-turbo-1106", t = 1.0)
                if c % 15 == 0:
                    print("summary finished: "+  str(c))

                path_src_list.append(filepath)
                dir_dst_list.append(dir_parent)
                # print(summary)
                # print(completion_dict)
                # with open(dst_path,'w') as f: ### write the 
                #     pass
            # print(dir_grand)
            # print(dir_parent)
            # print(dst_path)

Tokens in prompt:  4472
Tokens in completion:  395
Tokens in prompt:  9728
Tokens in completion:  360
Tokens in prompt:  4354
Tokens in completion:  385
Tokens in prompt:  5565
Tokens in completion:  354
Tokens in prompt:  3987
Tokens in completion:  374
Tokens in prompt:  7768
Tokens in completion:  432
Tokens in prompt:  7474
Tokens in completion:  371
Tokens in prompt:  3166
Tokens in completion:  346
Tokens in prompt:  4843
Tokens in completion:  371
Tokens in prompt:  12024
Tokens in completion:  365
Tokens in prompt:  3665
Tokens in completion:  479
Tokens in prompt:  6343
Tokens in completion:  353
Tokens in prompt:  5120
Tokens in completion:  405
Tokens in text:  81363
Text too long: Longer than 14000 tokens. Returning empty string and empty dictionary.
Tokens in prompt:  5165
Tokens in completion:  308
summary finished: 15
Tokens in prompt:  6585
Tokens in completion:  350
Tokens in text:  19398
Text too long: Longer than 14000 tokens. Returning empty string and empty diction

In [74]:
SUM_DIR = './spain_qr_results/RESOLUCION_DE_PROCEDIMIENTO_SANCIONADOR/'

In [76]:
"""
HERE PRODUCE THE JSON FROM THE SUMMARY
"""
res_list = []
comp_list = []
c = 0
for root, dirs, files in os.walk(SUM_DIR):
    for f in files:
        if f == "summary_t1.0.txt":
            c +=1
            filepath = os.path.join(root, f)
            summa = read_txt(filepath)
            path = pathlib.PurePath(filepath)
            parent = path.parent.name
            # grandpa = path.parent.parent.name
            save_dir = SUM_DIR + parent
            msg = construct_message_summary_to_json(summa, msg_sys_json, examples_summary, examples_json)
            res_dict, comp = summary_to_json(msg, save_dir)
            res_list.append(res_dict)
            comp_list.append(comp)
            if c % 15 == 0:
                print(" ----- JSON FINISHED: "+  str(c) + " -----")
                # print(summary)
                # print(completion_dict)
                # with open(dst_path,'w') as f: ### write the 
                #     pass
            # print(dir_grand)
            # print(dir_parent)
            # print(dst_path)
print(c)

Tokens in prompt:  4340
Tokens in completion:  337
Tokens in prompt:  4305
Tokens in completion:  374
Tokens in prompt:  4330
Tokens in completion:  414
Tokens in prompt:  4299
Tokens in completion:  405
Tokens in prompt:  4311
Tokens in completion:  369
Tokens in prompt:  4376
Tokens in completion:  416
Tokens in prompt:  4311
Tokens in completion:  434
Tokens in prompt:  4284
Tokens in completion:  342
Tokens in prompt:  4316
Tokens in completion:  334
Tokens in prompt:  4310
Tokens in completion:  439
Tokens in prompt:  4424
Tokens in completion:  367
Tokens in prompt:  4298
Tokens in completion:  359
Tokens in prompt:  4350
Tokens in completion:  405
Tokens in prompt:  4253
Tokens in completion:  321
Tokens in prompt:  4295
Tokens in completion:  361
 ----- JSON FINISHED: 15 -----
Tokens in prompt:  4296
Tokens in completion:  410
Tokens in prompt:  4328
Tokens in completion:  411
Tokens in prompt:  4294
Tokens in completion:  377
Tokens in prompt:  4325
Tokens in completion:  378


In [82]:
TRG_DIR_2 = './spain_qr_results/'

In [83]:
#### second part, for files in other categories
for root, dirs, files in os.walk(SRC_DIR):
    for f in files:
        if f.endswith(".pdf.es.txt"):
            filepath = os.path.join(root, f)
            txt = read_txt(filepath)
            path = pathlib.PurePath(filepath)
            parent = path.parent.name
            grandpa = path.parent.parent.name
            dir_grand  = TRG_DIR_2+grandpa
            dir_parent = TRG_DIR_2+grandpa+"/"+parent
                ### produced summary (completion dict + summary.txt)
                ### 
                ### change summary to json (need to prepare 3-4 examples.)
            if grandpa != "RESOLUCION_DE_PROCEDIMIENTO_SANCIONADOR":
                c +=1
                # print(filepath)
                # print(dir_parent)
                # print(txt)
                # sum_path = dir_parent + '/summary.txt'
                doc_path = dir_parent + '/document.txt'
                # if not os.path.exists(doc_path):
                with open(filepath, 'r', encoding="utf-8") as f:
                    original_text = f.read()

                with open(doc_path, "w") as f:
                    f.write(original_text)
                
                summary, completion_dict = write_summary(txt,dir_parent,country="spain", save_result = True,token_limit = 14000, model_name = "gpt-3.5-turbo-1106", t = 1.0)
                if c % 15 == 0:
                    print("summary finished: "+  str(c))

                path_src_list.append(filepath)
                dir_dst_list.append(dir_parent)
                # print(summary)
                # print(completion_dict)
                # with open(dst_path,'w') as f: ### write the 
                #     pass
            # print(dir_grand)
            # print(dir_parent)
            # print(dst_path)

Tokens in prompt:  6538
Tokens in completion:  375
Tokens in prompt:  3806
Tokens in completion:  329
Tokens in prompt:  6100
Tokens in completion:  417
Tokens in prompt:  9654
Tokens in completion:  480
summary finished: 900
Tokens in prompt:  8606
Tokens in completion:  479
Tokens in prompt:  6904
Tokens in completion:  360
Tokens in prompt:  6762
Tokens in completion:  420
Tokens in prompt:  7362
Tokens in completion:  528
Tokens in prompt:  6639
Tokens in completion:  409
Tokens in prompt:  6829
Tokens in completion:  428
Tokens in prompt:  5910
Tokens in completion:  386
Tokens in prompt:  5419
Tokens in completion:  417
Tokens in prompt:  7154
Tokens in completion:  444
Tokens in prompt:  8071
Tokens in completion:  566
Tokens in prompt:  7071
Tokens in completion:  455
Tokens in prompt:  7024
Tokens in completion:  428
Tokens in prompt:  5810
Tokens in completion:  454
Tokens in prompt:  5559
Tokens in completion:  55
Tokens in prompt:  5562
Tokens in completion:  398
summary fi

In [85]:
SUM_DIR = './spain_qr_results/'

In [90]:
"""
HERE PRODUCE THE JSON FROM THE SUMMARY
"""
c = 0 
for root, dirs, files in os.walk(SUM_DIR):
    for f in files:
        if f == "summary_t1.0.txt":
            c +=1
            filepath = os.path.join(root, f)
            summa = read_txt(filepath)
            path = pathlib.PurePath(filepath)
            parent = path.parent.name
            grandpa = path.parent.parent.name
            if grandpa != "RESOLUCION_DE_PROCEDIMIENTO_SANCIONADOR":
                save_dir = SUM_DIR + grandpa + "/" + parent
                msg = construct_message_summary_to_json(summa, msg_sys_json, examples_summary, examples_json)
                res_dict, comp = summary_to_json(msg, save_dir)
                res_list.append(res_dict)
                comp_list.append(comp)
                if c % 15 == 0:
                    print(" ----- JSON FINISHED: "+  str(c) + " -----")
                    print(summary)
                    print(completion_dict)
                    # with open(dst_path,'w') as f: ### write the 
                    #     pass
            # print(dir_grand)
            # print(dir_parent)
            # print(dst_path)
print(c)

Tokens in prompt:  4320
Tokens in completion:  409
Tokens in prompt:  4274
Tokens in completion:  345
Tokens in prompt:  4354
Tokens in completion:  439
Tokens in prompt:  4425
Tokens in completion:  410
Tokens in prompt:  4424
Tokens in completion:  443
Tokens in prompt:  4305
Tokens in completion:  364
Tokens in prompt:  4356
Tokens in completion:  421
Tokens in prompt:  4473
Tokens in completion:  415
Tokens in prompt:  4352
Tokens in completion:  382
Tokens in prompt:  4373
Tokens in completion:  359
Tokens in prompt:  4331
Tokens in completion:  425
Tokens in prompt:  4353
Tokens in completion:  393
Tokens in prompt:  4389
Tokens in completion:  410
Tokens in prompt:  4511
Tokens in completion:  380
Tokens in prompt:  4400
Tokens in completion:  404
 ----- JSON FINISHED: 15 -----
1. The document is a termination resolution of a sanctioning procedure issued by the Spanish Data Protection Agency (Agencia Española de Protección de Datos). It identifies the procedure number as PS/0019